In [1]:
# Sentiment analysis using Amazon reviews - Ernest Leem
# https://erleem.medium.com/nlp-complete-sentiment-analysis-on-amazon-reviews-374e4fea9976
# runs correctly -egs-


In [1]:

import os, json, gzip 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gzip

import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet, stopwords

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/suchanek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/suchanek/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/suchanek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /Users/suchanek/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/suchanek/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
#!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz


In [3]:

data = []
with gzip.open('reviews_Electronics_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
df = pd.DataFrame.from_dict(data)

In [6]:
# drop any rows w/ missing values
df = df.dropna()
# discover the actual counts
df.overall.value_counts()
# set sample size to labels w/ minimum count
sample_size = 50000
df_equal_overall = pd.DataFrame()
for i in df.overall.unique():
  X = df[df.overall == i].sample(sample_size)
  df_equal_overall = df_equal_overall.append(X)

/var/folders/3x/846n4xzj073b5sjsmq1f0z3c0000gp/T/ipykernel_34086/878245883.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_equal_overall = df_equal_overall.append(X)
/var/folders/3x/846n4xzj073b5sjsmq1f0z3c0000gp/T/ipykernel_34086/878245883.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_equal_overall = df_equal_overall.append(X)
/var/folders/3x/846n4xzj073b5sjsmq1f0z3c0000gp/T/ipykernel_34086/878245883.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_equal_overall = df_equal_overall.append(X)
/var/folders/3x/846n4xzj073b5sjsmq1f0z3c0000gp/T/ipykernel_34086/878245883.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

In [7]:
stopwords_list = stopwords.words('english')

def ReviewProcessing(df):
  # remove non alphanumeric 
  df['review_cleaned'] = df.reviewText.str.replace('[^a-zA-Z0-9 ]', '')
  # lowercase
  df.review_cleaned = df.review_cleaned.str.lower()
  # split into list
  df.review_cleaned = df.review_cleaned.str.split(' ')
  # remove stopwords
  df.review_cleaned = df.review_cleaned.apply(lambda x: [item for item in x if item not in stopwords_list])
  return df

In [8]:
def get_wordnet_pos(word):
  tag = nltk.pos_tag([word])[0][1][0].upper()
  tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

  return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = nltk.stem.WordNetLemmatizer()
def get_lemmatize(sent):
  return " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sent)])

In [9]:
clean_data = ReviewProcessing(df_equal_overall)
clean_data.review_cleaned = clean_data.review_cleaned.apply(' '.join)
clean_data['review_cleaned_lemmatized'] = clean_data.review_cleaned.apply(get_lemmatize)


/var/folders/3x/846n4xzj073b5sjsmq1f0z3c0000gp/T/ipykernel_34086/2005400237.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['review_cleaned'] = df.reviewText.str.replace('[^a-zA-Z0-9 ]', '')


In [8]:
from sklearn.naive_bayes import MultinomialNB

nb = Pipeline([('vectorize', CountVectorizer(ngram_range=(1, 2))),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [9]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier()),
               ])


In [10]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(max_iter=500)),
               ])

In [11]:
x = clean_data['review_cleaned_lemmatized']
y = clean_data['overall']
X_train, X_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size=0.2, stratify=y, random_state = 44)


In [12]:
# Naive Bayes
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
print(accuracy_score(y_test, y_pred_nb))
print(confusion_matrix(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))

# SGD Classifier
sgd.fit(X_train, y_train)
y_pred_sgd = sgd.predict(X_test)
print(accuracy_score(y_test, y_pred_sgd))
print(confusion_matrix(y_test, y_pred_sgd))
print(classification_report(y_test, y_pred_sgd))

# Logistic Regression
logreg.fit(X_train, y_train)
y_pred_log = logreg.predict(X_test)
print(accuracy_score(y_test, y_pred_log))
print(confusion_matrix(y_test, y_pred_log))
print(classification_report(y_test, y_pred_log))

0.46084
[[5762 3001  766  361  110]
 [2459 4449 2003  962  127]
 [1092 2555 3393 2695  265]
 [ 532 1066 1593 5731 1078]
 [ 579  664  689 4361 3707]]
              precision    recall  f1-score   support

         1.0       0.55      0.58      0.56     10000
         2.0       0.38      0.44      0.41     10000
         3.0       0.40      0.34      0.37     10000
         4.0       0.41      0.57      0.48     10000
         5.0       0.70      0.37      0.48     10000

    accuracy                           0.46     50000
   macro avg       0.49      0.46      0.46     50000
weighted avg       0.49      0.46      0.46     50000

0.4744
[[8063  665  410  260  602]
 [5009 1905 1333  757  996]
 [2124 1396 2684 2000 1796]
 [ 788  461 1025 3173 4553]
 [ 624  165  333  983 7895]]
              precision    recall  f1-score   support

         1.0       0.49      0.81      0.61     10000
         2.0       0.41      0.19      0.26     10000
         3.0       0.46      0.27      0.34     100

In [13]:
from sklearn.model_selection import GridSearchCV

grid=[{'clf__solver': ['lbfgs', 'sag', 'saga'],
       'clf__C': [0.01, 0.1, 1]}]
lr = GridSearchCV(logreg, param_grid = grid, cv = 5, scoring='accuracy', verbose = 1, n_jobs = -1)
best_model = lr.fit(X_train, y_train)

print(best_model.best_estimator_)
print(best_model.best_score_)

y_pred_grid = best_model.predict(X_test)
print(confusion_matrix(y_test, y_pred_grid))
print(classification_report(y_test, y_pred_grid))
print(accuracy_score(y_test, y_pred_grid))

Fitting 5 folds for each of 9 candidates, totalling 45 fits


In [ ]:
print('All Done!')